Exercise from Think Stats, 2nd Edition (thinkstats2.com)<br>
Allen Downey

In [32]:
%matplotlib inline

import nsfg
import chap01soln
import statsmodels.formula.api as smf

Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool. 

In [33]:
data = nsfg.ReadFemPreg()
data = data[(data.prglngth > 30) & (data.outcome == 1)]
resp = chap01soln.ReadFemResp()
resp.index = resp.caseid
joined = data.join(resp, on='caseid', rsuffix='_r')

joined.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,pubassis_i_r,basewgt_r,adj_mod_basewgt_r,finalwgt_r,secu_r,sest_r,cmintvw_r,cmlstyr,screentime,intvlngth
0,1,1,NaN,NaN,NaN,NaN,6,NaN,1,NaN,...,0,3410.389399,3869.349602,6448.271112,2,9,1231,1219,19:56:43,67.563833
1,1,2,NaN,NaN,NaN,NaN,6,NaN,1,NaN,...,0,3410.389399,3869.349602,6448.271112,2,9,1231,1219,19:56:43,67.563833
2,2,1,NaN,NaN,NaN,NaN,5,NaN,3,5,...,0,7226.301740,8567.549110,12999.542264,2,12,1231,1219,14:54:03,106.018167
3,2,2,NaN,NaN,NaN,NaN,6,NaN,1,NaN,...,0,7226.301740,8567.549110,12999.542264,2,12,1231,1219,14:54:03,106.018167
4,2,3,NaN,NaN,NaN,NaN,6,NaN,1,NaN,...,0,7226.301740,8567.549110,12999.542264,2,12,1231,1219,14:54:03,106.018167


In [34]:
# Following code taken from ThinkStats and annotated to show understanding
t = []
for col in joined.columns:
    try:
        # Skip any columns with tiny variance since they won't tell us much
        if joined[col].var() < 1e-7:
            continue

        # Create a model to examine how the baby's weight is related to 
        # the mother's age and the current column
        formula = 'totalwgt_lb ~ agepreg + ' + col
        model = smf.ols(formula, data=joined)
        
        # If less than half of the rows were used, skip this column
        if model.nobs < len(joined)/2:
            continue

        results = model.fit()
    except (ValueError, TypeError):
        continue

    t.append((results.rsquared, col))

# Sort the results by R-squared and display the top 30
t.sort(reverse=True)
for mse, name in t[:30]:
    print(name, mse)

('totalwgt_lb', 1.0)
(u'birthwgt_lb', 0.94981273059780091)
(u'lbw1', 0.30082407844707948)
(u'prglngth', 0.13012519488625029)
(u'wksgest', 0.12340041363360821)
(u'agecon', 0.1020314992815573)
(u'mosgest', 0.027144274639580579)
(u'babysex', 0.018550925293941312)
(u'race_r', 0.016199503586251329)
(u'race', 0.016199503586251329)
(u'nbrnaliv', 0.016017752709791666)
(u'paydu', 0.014003795578110934)
(u'rmarout03', 0.013430066465712986)
(u'birthwgt_oz', 0.013102457615710938)
(u'anynurse', 0.012529022541812429)
(u'bfeedwks', 0.012193688404499081)
(u'totincr', 0.011870069031174046)
(u'marout03', 0.011807801994371481)
(u'marcon03', 0.011752599354397653)
(u'cebow', 0.011437770919637935)
(u'rmarout01', 0.011407737138643737)
(u'rmarout6', 0.011354138472803199)
(u'marout01', 0.011269357246801448)
(u'hisprace_r', 0.011238349302033601)
(u'hisprace', 0.011238349302033601)
(u'mar1diss', 0.010961563590753065)
(u'fmarcon5', 0.0106049646843005)
(u'rmarout02', 0.010546913206563868)
(u'marcon02', 0.0104814017

The top three columns are related to the weight of the baby, so we obviously can't use them in our prediction.  Three of the next four columns are related to the length of the pregnancy which is also unknown until after the fact.  The other column is the mother's age at conception, but this seems redundant since we're already using the mother's age at the end of the pregnancy.  The sex of the baby is pretty high on the list and is most likely known by the 30th week of the pregnancy.  Race is the next highest factor, but as the reading mentions, can sometimes be coupled with various socioeconomic factors.  Out of the remaining columns, number of babies in the pregnancy, whether the mother owns or rents a home, and the mother's total family income could be known or reasonably estimated.

In [35]:
# Model from ThinkStats
formula = ('totalwgt_lb ~ agepreg + C(race) + babysex==1 + '
               'nbrnaliv>1 + paydu==1 + totincr')
results = smf.ols(formula, data=joined).fit()
print results.rsquared

# My modified model
formula = ('totalwgt_lb ~ agepreg + C(race) + babysex==1 + '
               'C(nbrnaliv) + paydu==1 + totincr')
results = smf.ols(formula, data=joined).fit()
print results.rsquared

0.0599863083304
0.0618127968117


Taking the model that Allen described in _ThinkStats_, I decided to make a slight modification regarding how multiple babies were handled.  My hypothesis was that I would be able to get better results by looking at this categorically (how many babies are in the pregnancy?) rather than as a boolean (is there more than one baby in the pregnancy?).  My reasoning is that it seems to me that triplets would each weigh less than twins who would each weigh less than a single child, and the specific number of babies in the pregnancy would probably be known.  The slight improvment in R$^2$ value seems to indicate that being more specific about this factor would improve your odds in guessing the weight of the baby/babies (though it's still not that much of an improvement over guessing without taking these factors into account).

## Clarifying Questions

Use this space to ask questions regarding the content covered in the reading. These questions should be restricted to helping you better understand the material. For questions that push beyond what is in the reading, use the next answer field. If you don't have a fully formed question, but are generally having a difficult time with a topic, you can indicate that here as well.

It seems like a lot of factor can be coupled, such as home ownership status and annual income.  Are there any pitfalls to be aware of regarding this?  Does it matter at all when doing regressions?

## Enrichment Questions

Use this space to ask any questions that go beyond (but are related to) the material presented in this reading. Perhaps there is a particular topic you'd like to see covered in more depth. Perhaps you'd like to know how to use a library in a way that wasn't show in the reading. One way to think about this is what additional topics would you want covered in the next class (or addressed in a followup e-mail to the class). I'm a little fuzzy on what stuff will likely go here, so we'll see how things evolve.

## Additional Resources / Explorations

If you found any useful resources, or tried some useful exercises that you'd like to report please do so here. Let us know what you did, what you learned, and how others can replicate it.